In [10]:
!docker stop ipfs_host

ERROR: The Compose file './docker-compose.yml' is invalid because:
Unsupported config option for services.networks: 'backend'


In [11]:
!docker rm ipf_host

ERROR: The Compose file './docker-compose.yml' is invalid because:
Unsupported config option for services.networks: 'backend'


In [1]:
!docker run -d --name ipfs_host2 -v $ipfs_staging:/export -v $ipfs_data:/data/ipfs -p 4001:4001 -p 4001:4001/udp -p 127.0.0.1:8080:8080 -p 127.0.0.1:5001:5001 ipfs/go-ipfs:latest

2c7f43c59f574aca03a3b47d20ba69f76063d5b29eb8e2d87ba16786d3d42fb4


In [25]:
!mkdir /tmp/posts
!mkdir /tmp/comments

/home/chia/miniconda3/lib/python3.8/site-packages/ipfshttpclient/client/__init__.py:73: VersionMismatch: Unsupported daemon version '0.13.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import json
import sqlite3
import pandas as pd
import substrateinterface
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.exceptions import SubstrateRequestException
from substrate_helpers import make_call, addSchema, get_msa_id, \
            get_signature, create_msa_with_delegator, mint_votes, mint_user, get_schemas_from_pattern, \
            get_content_from_schemas, get_user, follow_user, add_delegate, mint_data

In [28]:
path = '/tmp/'
bob = Keypair.create_from_uri('//Bob')
substrate = SubstrateInterface(
    url="ws://127.0.0.1:9944",
    ss58_format=42,
    type_registry_preset='kusama'
)

In [29]:
bob_msa_id = get_msa_id(bob, create=True)

In [30]:
bob_msa_id

In [31]:
con = sqlite3.connect('postthreadV1_write.db')
cur = con.cursor()

# Make schemas

In [32]:
post_schema = "category STRING,title STRING,body STRING,url STRING,is_nsfw NUMERIC"
post_schemaId = addSchema(post_schema, wait_for_inclusion=True, wait_for_finalization=False)

comment_schema = "post_hash STRING,parent_hash STRING,depth NUMERIC,body STRING"
comment_schemaId = addSchema(comment_schema, wait_for_inclusion=True, wait_for_finalization=False)

vote_schema = "post_hash STRING,parent_hash STRING,parent_type STRING,num_votes NUMERIC"
vote_schemaId = addSchema(vote_schema, wait_for_inclusion=True, wait_for_finalization=False)

user_schema = "msa_id NUMERIC,username STRING,password STRING,profile_pic STRING,wallet_ss58_address STRING"
user_schemaId = addSchema(user_schema, wait_for_inclusion=True, wait_for_finalization=False)

follow_schema = "protagonist_msa_id NUMERIC,antagonist_msa_id NUMERIC,event STRING"
follow_schemaId = addSchema(follow_schema, wait_for_inclusion=True, wait_for_finalization=False)

link_schema = "account_type STRING,account_value STRING"
link_schemaId = addSchema(link_schema, wait_for_inclusion=True, wait_for_finalization=False)

payout_schema = "payout_amount NUMERIC"
payout_schemaId = addSchema(payout_schema, wait_for_inclusion=True, wait_for_finalization=False)

In [33]:
schemas = {
    "post": post_schemaId, "comment": comment_schemaId, 
    "vote": vote_schemaId, "user": user_schemaId, "follow": follow_schemaId, 
    "link": link_schemaId, "payout": payout_schemaId,
}
json.dump(schemas, open("schemas.json", "w"))

In [34]:
schemas

{'post': 1,
 'comment': 2,
 'vote': 3,
 'user': 4,
 'follow': 5,
 'link': 6,
 'payout': 7}

# Mint accounts 

In [35]:
accounts = {}
for account in ['Alice', 'Charlie', 'Dave', 'Eve', 'Ferdie']:
    wallet = Keypair.create_from_uri(f'//{account}password')
    user_msa_id = create_msa_with_delegator(bob, wallet)
    receipt_user = mint_user(user_msa_id, account, "password", "profile_pic", wallet)
    accounts[account] = user_msa_id

In [36]:
json.dump(accounts, open("accounts.json", "w"))

In [37]:
accounts

{'Alice': 2, 'Charlie': 3, 'Dave': 4, 'Eve': 5, 'Ferdie': 6}

# Have accounts follow each other

In [38]:
for name1, k1 in accounts.items():
    for name2, k2 in accounts.items():
        if k1 != k2:
            print(name1, name2)
            receipt = follow_user(k1, k2, True, wait_for_inclusion=False, wait_for_finalization=False)

Alice Charlie
Alice Dave
Alice Eve
Alice Ferdie
Charlie Alice
Charlie Dave
Charlie Eve
Charlie Ferdie
Dave Alice
Dave Charlie
Dave Eve
Dave Ferdie
Eve Alice
Eve Charlie
Eve Dave
Eve Ferdie
Ferdie Alice
Ferdie Charlie
Ferdie Dave
Ferdie Eve


In [39]:
receipt = follow_user(accounts['Dave'], accounts['Eve'], False, wait_for_inclusion=False, wait_for_finalization=False)
receipt = follow_user(accounts['Eve'], accounts['Dave'], False, wait_for_inclusion=False, wait_for_finalization=False)
receipt = follow_user(accounts['Dave'], accounts['Eve'], True, wait_for_inclusion=False, wait_for_finalization=False)

In [40]:
wallet = Keypair.create_from_uri(f'//the_doge_fatherpassword')
user_msa_id = create_msa_with_delegator(bob, wallet)
receipt_user = mint_user(user_msa_id, account, "password", "profile_pic", wallet)
for name1, k1 in accounts.items():
    receipt = follow_user(k1, user_msa_id, True, wait_for_inclusion=False, wait_for_finalization=False)

# Test post

In [41]:
post_data = {
    "category": "test",
    "title": "test title",
    "body": "test post",
    "url": "",
    "is_nsfw": False
}

username = 'Charlie'
password = 'password'

user_wallet = Keypair.create_from_uri('//' + username + password)
user_msa_id = get_msa_id(user_wallet)

post_data_hash, receipt_post = mint_data(post_data, user_msa_id, schemas['post'], path+'posts/', 
                                         wait_for_inclusion=True, wait_for_finalization=False)

In [42]:
receipt_post.error_message

In [43]:
example_post = pd.read_sql_query(f"SELECT ipfs_hash FROM post WHERE msa_id_from_query = {accounts['Charlie']} LIMIT 1", con)['ipfs_hash'].iloc[0]
example_post

IndexError: single positional indexer is out-of-bounds

In [ ]:
comment_data = {
    "post_hash": example_post,
    "parent_hash": example_post,
    "depth": 0,
    "body": "example comment 2",
}
username = 'Charlie'
password = 'password'

user_wallet = Keypair.create_from_uri('//' + username + password)
user_msa_id = get_msa_id(user_wallet)

comment_data_hash, receipt_comment = mint_data(comment_data, user_msa_id, schemas['comment'], path+'comments/', 
                                               wait_for_inclusion=True, wait_for_finalization=False)


In [ ]:
account_type = "gmail"
account_value = "example@gmail.com"
link_data = '{' + f'"account_type": "{account_type}","account_value": "{account_value}"' + '}'

username = 'Charlie'
password = 'password'

user_wallet = Keypair.create_from_uri('//' + username + password)
user_msa_id = get_msa_id(user_wallet)

link_data_hash, receipt_link = mint_data(link_data, user_msa_id, schemas['link'], 
                                               wait_for_inclusion=True, wait_for_finalization=False)

In [ ]:
receipt_link.error_message

In [ ]:
    data = '{' + f'"post_hash": "{example_post}","parent_hash": "{example_post}","parent_type": "post","num_votes": 1' + '}'
    _, receipt = mint_data(data, user_msa_id, schemas['vote'], 
                        wait_for_inclusion=True, wait_for_finalization=False)
    

In [ ]:

    payout_amount = 1000
    receipt = make_call("Balances", "transfer", {"dest": "5FqWBycSL7R7bG3VNRRHftv98kCjGCQ3oPT34tMUTwfsvcUj", "value": payout_amount}, bob, 
                        wait_for_inclusion=True, wait_for_finalization=False)
    data = '{' + f'"payout_amount": {payout_amount}' + '}'
    mint_data(data, user_msa_id, schemas['payout'], wait_for_inclusion=True, wait_for_finalization=False)